# Analysis of Sales Department Efficiency:
1. Evaluate the efficiency of individual deal owners and advertising campaigns in terms of the number of processed deals, conversion rate, and total sales amount.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
deals = pd.read_pickle('deals_df.pkl')
spend = pd.read_pickle('spend_df.pkl')
calls = pd.read_pickle('calls_df.pkl')

# Comparison of Successful and Lost Deals

In [ ]:
# Data Preparation: grouping deals by campaigns
campaign_data = deals.groupby('Deal Owner Name').agg(
total_leads=('Id', 'count'),  # Total number of leads
completed_leads=('Stage', lambda x: (x == 'Payment Done').sum()),  # Completed leads
failed_leads=('Stage', lambda x: (x == 'Lost').sum()),  # Lost leads
total_sales=('Initial Amount Paid', 'sum'),
total_offer_amount=('Offer Total Amount', 'sum')).reset_index()

campaign_data['total_offer_amount'] = campaign_data['total_offer_amount'].astype(int)
campaign_data['total_sales'] = campaign_data['total_sales'].astype(int)
campaign_data['conversion_rate'] = campaign_data['completed_leads'] / campaign_data['total_leads'] * 100
# len(campaign_data)
final_campaign_data = campaign_data.sort_values('total_leads', ascending=False)

In [ ]:
plt.figure(figsize=(10, 8))
ax = plt.gca()

# Columns for total number of leads
bars = ax.bar(final_campaign_data['Deal Owner Name'], final_campaign_data['total_leads'], color='brown', label='Total number of deals')

# Lines for completed and lost leads
ax.plot(final_campaign_data['Deal Owner Name'], final_campaign_data['completed_leads'], color='forestgreen', marker='o', linestyle='-', label='Successful deals')
ax.plot(final_campaign_data['Deal Owner Name'], final_campaign_data['failed_leads'], color='coral', marker='o', linestyle='-.', label='Lost deals')

# Adding numbers above the bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height, f'{int(height)}', ha='center', va='bottom', fontsize=9)

# Setting up the graph
ax.set_title('Comparison of Successful and Lost Deals')
ax.set_xlabel('Deal Owner Name')
ax.set_ylabel('Number of deals')
ax.tick_params(axis='x', rotation=90)
ax.legend(loc='best')

# Optimization and display
plt.tight_layout()
plt.show()

# Conversion Chart by Employees

In [ ]:
# Grouping data by sources
source_data = deals.groupby('Deal Owner Name', observed=True).agg(
    total_leads=('Id', 'count'),
    completed_leads=('Stage', lambda x: (x == 'Payment Done').sum())
).reset_index()

# Calculating the conversion percentage
source_data['Completed_leads(%)'] = (source_data['completed_leads'] / source_data['total_leads']) * 100
average_conversion = source_data['Completed_leads(%)'].mean()

# Sorting by total number of leads in descending order
source_data = source_data.sort_values('total_leads', ascending=False)

# Selecting the 10 most successful campaigns
top_10_campaigns = source_data.head(20)

# Creating the chart
plt.figure(figsize=(10, 8))
ax = plt.gca()

# Bars for the total number of leads
ax.bar(top_10_campaigns['Deal Owner Name'], top_10_campaigns['total_leads'], color='brown', label='Total Deals')
# Bars for successful leads (overlay)
ax.bar(top_10_campaigns['Deal Owner Name'], top_10_campaigns['completed_leads'], color='orange', label='Completed Deals')

# Line for the conversion percentage
ax2 = ax.twinx()
ax2.plot(top_10_campaigns['Deal Owner Name'], top_10_campaigns['Completed_leads(%)'], color='blue', marker='o', linestyle='-.', label='Conversion Percentage')

# Adding numbers above the bars
for i, v in enumerate(top_10_campaigns['total_leads']):
    ax.text(i, v + 10, str(int(v)), ha='center')
for i, v in enumerate(top_10_campaigns['completed_leads']):
    ax.text(i, v + 10, str(int(v)), ha='center', color='white')

# Setting up the chart
ax.set_title('Conversion by Employees')
ax.set_xlabel('')
ax.set_ylabel('Number of Deals')
ax.tick_params(axis='x', rotation=90)
ax.legend(loc='center left', bbox_to_anchor=(0.5, 0.95), ncol=1)

# Adding text with the average conversion
ax.text(0.5, max(top_10_campaigns['total_leads']) * 1.1, f'Average Conversion: {average_conversion:.2f}%', ha='center', fontsize=12, color='black')

ax2.legend(loc='center right', bbox_to_anchor=(0.5, 0.95), ncol=3)

# Optimization and display
plt.tight_layout()
plt.show()

# Chart of Total Sales by Managers and Top-10 Campaigns

# Heatmap of Manager Performance

In [ ]:
offer_amount_heatmap_data = deals[deals['Campaign'] != 'Unknown'].groupby(['Deal Owner Name', 'Campaign']).agg(
    total_offer_amount=('Offer Total Amount', 'sum')
).reset_index()

top_campaigns = offer_amount_heatmap_data.groupby('Campaign')['total_offer_amount'].sum().sort_values(ascending=False).head(10).index

filtered_owner_campaigns = offer_amount_heatmap_data[offer_amount_heatmap_data['Campaign'].isin(top_campaigns)]

offer_amount_heatmap_data = filtered_owner_campaigns.pivot_table(
    values='total_offer_amount',
    index='Deal Owner Name',
    columns='Campaign',
    aggfunc='sum',
    fill_value=0
)

offer_amount_heatmap_data = offer_amount_heatmap_data.loc[
    (offer_amount_heatmap_data.sum(axis=1) > 0),
    (offer_amount_heatmap_data.sum(axis=0) > 0)
]

plt.figure(figsize=(14, 8))
sns.heatmap(offer_amount_heatmap_data, annot=True, cmap='YlOrRd', fmt='.1f', cbar=True, linewidths=0.5)
plt.title('Total Sales Amount by Managers and Top 10 Campaigns')
plt.xticks(rotation=-45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Summary table of deal manager performance indicators

In [ ]:
# Grouping data by deal owners
campaign_data = deals.groupby('Deal Owner Name').agg(
    total_leads=('Id', 'count'),  # Total number of leads
    completed_leads=('Stage', lambda x: (x == 'Payment Done').sum()),  # Completed leads
    total_sales=('Initial Amount Paid', 'sum'),
    total_offer_amount=('Offer Total Amount', 'sum')
).reset_index()

# Calculating the percentage of completed deals
campaign_data['Completed_leads(%)'] = (campaign_data['completed_leads'] / campaign_data['total_leads']) * 100

# Sorting by descending Completed_leads(%)
campaign_data = campaign_data.sort_values('Completed_leads(%)', ascending=False)

# Formatting the table for display
styled_campaign_data = (campaign_data.style
    .hide(axis='index')
    .format({
        'total_leads': '{:,.0f}',
        'completed_leads': '{:,.0f}',
        'total_sales': '${:,.2f}',
        'total_offer_amount': '${:,.2f}',
        'Completed_leads(%)': '{:.1f}%'
    })
    .background_gradient(subset=['Completed_leads(%)', 'total_sales', 'total_offer_amount'], cmap='YlOrRd')
    .set_properties(**{'text-align': 'center'})
    .set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('text-align', 'center')]}
    ]).bar(subset=['total_leads', 'completed_leads'], color='#F4A261', align='mid')
)

display(styled_campaign_data)

# Conclusions
1. Effectiveness of Deal Owners
The analysis of the effectiveness of deal owners (Deal Owner Name) and advertising campaigns based on the provided data reveals key differences in performance. Deal owners handled between 1 and 2,963 leads, with conversion rates ranging from 0% to 30.7% and total sales amounts from $0 to $5,117,800.00.

- The most effective in terms of conversion was **Oliver Taylor** with 30.7% (50 successful deals out of 163 leads) and total sales of $152,650.00, highlighting his exceptional ability to close deals with a relatively small volume of leads.

- However, in terms of the number of leads processed, **Charlie Davis** (2,963 leads) and **Ulysses Adams** (2,165 leads) lead the way. Their conversion rates are 5% and 6.5%, respectively, with sales amounts of $3,822,500.00 and $5,117,800.00, making them leaders in overall contribution to total sales.

- Among the less active operators, such as **John Doe** (20 leads, 5% conversion) or **George King** (94 leads, 2.1% conversion), low sales amounts ($4,600.00 and $2,900.00, respectively) are noticeable, indicating limited effectiveness.

- Operators with zero conversion, such as **Amy Green**, **Alice Johnson**, **Sam Young**, **Rachel White**, and others, require a review of strategy, as they did not close any deals while processing leads. For example, **Rachel White** processed 871 leads but did not close a single deal, resulting in total sales of only $14,000.00.

#### Conclusion
The analysis showed that the most effective deal owners are **Oliver Taylor**, **Ulysses Adams**, and **Charlie Davis**, who demonstrate high results in both sales volume and conversion rate. However, there is significant potential for improvement among owners with low effectiveness, especially those who have not closed any deals.

Recommendations:
1. Conduct training for deal owners to enhance their lead management skills.
2. Optimize the lead distribution strategy to focus on more promising operators.
3. Analyze the reasons for low conversion rates among operators with a large number of leads, such as **Rachel White** and **Nina Scott**, to address potential issues.